In [1]:
import itertools as it
from gurobipy import *

## Wyndor_Case

In [2]:
m = Model("Wyndor_Case")

In [3]:
x1 = m.addVar(vtype=GRB.CONTINUOUS, name = "x1")
x2 = m.addVar(vtype=GRB.CONTINUOUS, name = "x2")

In [4]:
m.setObjective(3*x1 + 5*x2, GRB.MAXIMIZE)
m.addConstr(x1 <= 4, "c0")
m.addConstr(2*x2 <= 12, "c1")
m.addConstr(3*x1 + 2*x2 <= 18, "c2")
m.update()

In [5]:
m.optimize()

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  3.600000000e+01


In [6]:
m.printAttr("x", "x*")


    Variable            x 
-------------------------
          x1            2 
          x2            6 


# Larger problem - Blending, Oil refine and Food process Case

## Problem formulation

### Description
There is a company which **produces a certain product** by **refining 5 kinds of raw oils** (VEG1, VEG2, OIL1, OIL2, OIL3) and processing them. Here, VEG means vegetable based and OIL means non-vegetable based oil. The sales price is fixed (150) while the purchasing price of raw oils varies (rises/falls) over time. (see the table below.)

|Month|VEG1|VEG2|OIL1|OIL2|OIL3|
|-|-|-|-|-|
|January|110|120|130|110|115|
|February|130|130|110|90|115|
|March|110|140|130|100|95|
|May|100|120|150|110|105|
|June|90|100|140|80|135|

There is a certain **limit of amount the company can refine** the raw oils. It is upto 250 for vegetable oils (VEG1 and VEG2) and 200 for non-vegetable oils (OIL1, OIL2 and OIL3) respectively.

The company may **store the raw oils** purchased in order to refine them next month. In this case, the cost for the storage is 5. Every initial value of stored raw oil is 500 for all 5 oils and this value must stay as same at the end of June.

The company can also make its own decision about the **blending ratio of raw oils**, however a requirement does exist. It is that the **hardness** of the final product shall be between 3 and 6. The hardness of the final product can be linearly calculated from the raw oil used. The hardness of raw oil is also given below.

|Raw Oil|VEG1|VEG2|OIL1|OIL2|OIL3|
|-|-|-|-|-|
|Hardness|8.8|6.1|2.0|4.2|5.0|

Under the above conditions, the company wants to know the best plan/strategy/decision to maximize the profit.

### Conceptualisation & Formalisation
- OR is about "allocating limited resources to competing activities in optimal way which makes the objective most desirable."
- Always bear in mind: What activities, resources and objective are there?

In [7]:
# instantiate model
m = Model("Oil_food")

#### Decision Variables: Determine the Level of Actitivities
First, the company can 1) buy 2) refine and 3) store raw oils for each month. The level of activity (decision variable) means:
- which kind and how much raw oils to buy?
- which kind and how much raw oils to refine?
- which kind and how much raw oils to store? (for next month)

These 3 (buy/refine/store) activities are defined for every month (January, February, March, April, May and June.), for every oil (VEG1, VEG2, OIL1, OIL2, OIL3). Therefore, we may define 90 (5 kinds × 6 months × 3 activities) decision variables. 

$$B_{month, oil}$$
$$R_{month, oil}$$
$$S_{month, oil}$$
$Here,$

$$month \in \{Jan, Feb, Mar, Apr, May, Jun\}$$
$$oil \in \{VEG1, VEG2, OIL1, OIL2, OIL3\}$$

In [8]:
time_periods = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]
oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]

# add variables with using subscript
buy = m.addVars(time_periods, oils, name = "Buy")
refine = m.addVars(time_periods, oils, name = "Refine")
store = m.addVars(time_periods, oils, name = "Store")

Additionally, given the raw oils refined (with a certain combination), it can 4) produce the product for each month. Again the level of activity means:
- How much to produce?

This activity is also defined for every month, hence 6 additioanl decision variables are defined.

$$P_{month}$$

In [9]:
produce = m.addVars(time_periods, name = "Produce")
m.update()

#### Constraints
Second, what kind of resources the company has? (they serve as constraints.) There are some of them which are explicitly given:
- The capacity of refining (200, 250)
- Hardness requirement (3 ≤ hardness ≤ 6)
- Initial and Final storage

(For each month) the sum of refining vegetable oils (VEG1, 2) shouldn't exceed 200. Similarly, the sum of non-vegetable oils (OIL1, 2, 3) shouldn't exceed 250. For example, the refine in January for VEG1 and VEG2 cannot exceeed 200.

$$R_{Jan, VEG1} + R_{Jan, VEG2} \leq 200$$

In [10]:
vegLim = 200
c11 = (quicksum(refine[time_period, oil] for oil in oils if "VEG" in oil) <= vegLim for time_period in time_periods)
m.addConstrs(c11, "vegLim")
list((quicksum(refine[time_period, oil] for oil in oils if "VEG" in oil) <= vegLim for time_period in time_periods))

[<gurobi.TempConstr: <gurobi.LinExpr: Refine[Jan,VEG1] + Refine[Jan,VEG2]> <= 200>,
 <gurobi.TempConstr: <gurobi.LinExpr: Refine[Feb,VEG1] + Refine[Feb,VEG2]> <= 200>,
 <gurobi.TempConstr: <gurobi.LinExpr: Refine[Mar,VEG1] + Refine[Mar,VEG2]> <= 200>,
 <gurobi.TempConstr: <gurobi.LinExpr: Refine[Apr,VEG1] + Refine[Apr,VEG2]> <= 200>,
 <gurobi.TempConstr: <gurobi.LinExpr: Refine[May,VEG1] + Refine[May,VEG2]> <= 200>,
 <gurobi.TempConstr: <gurobi.LinExpr: Refine[Jun,VEG1] + Refine[Jun,VEG2]> <= 200>]

In [11]:
# same for non-vegetable oils
oilLim = 250
c12 = (quicksum(refine[time_period, oil] for oil in oils if "OIL" in oil) <= oilLim for time_period in time_periods)
m.addConstrs(c12, "oilLim")

{'Apr': <gurobi.Constr *Awaiting Model Update*>,
 'Feb': <gurobi.Constr *Awaiting Model Update*>,
 'Jan': <gurobi.Constr *Awaiting Model Update*>,
 'Jun': <gurobi.Constr *Awaiting Model Update*>,
 'Mar': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>}

And when the company produces, the result of blending 5 raw oils shall meet the hardness requirement (between 3 and 6) For example, the product of January the following condition should be met.

$$3 \times P_{Jan} \leq 8.8 \times R_{Jan, VEG1} + 6.1 \times R_{Jan, VEG2} + 2.0 \times R_{Jan, OIL1} + 4.2 \times R_{Jan, OIL2} + 5.0 \times R_{Jan, OIL3}\} \leq 6 \times P_{Jan}$$

In [12]:
hardness = [8.8, 6.1, 2.0, 4.2, 5.0]
hardness = dict(zip(oils, hardness))
hardLower = 3
hardUpper = 6

c21 = (
    quicksum(hardness[oil] * refine[time_period, oil] for oil in oils) 
    >= hardLower*produce[time_period] for time_period in time_periods
)
c22 = (
    quicksum(hardness[oil] * refine[time_period, oil] for oil in oils) 
    <= hardUpper*produce[time_period] for time_period in time_periods
)
m.addConstrs(c21, "hardLower")
m.addConstrs(c22, "hardUpper")

{'Apr': <gurobi.Constr *Awaiting Model Update*>,
 'Feb': <gurobi.Constr *Awaiting Model Update*>,
 'Jan': <gurobi.Constr *Awaiting Model Update*>,
 'Jun': <gurobi.Constr *Awaiting Model Update*>,
 'Mar': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>}

Plus, the initial and final storage gives following constraints.
$$500 + B_{Jan, oil} = R_{Jan, oil} + S_{Jan, oil}$$
$$S_{May, oil} + B_{May, oil} = R_{May, oil} + 500$$

In [13]:
initStore = 500
lastStore = initStore

c31 = (initStore + buy[time_periods[0], oil] ==
     refine[time_periods[0], oil] + store[time_periods[0], oil]
     for oil in oils)
m.addConstrs(c31, "initStore")
c32 = (
    store[time_periods[-1], oil]  == lastStore
    for oil in oils
)
m.addConstrs(c32, "lastStore")

{'OIL1': <gurobi.Constr *Awaiting Model Update*>,
 'OIL2': <gurobi.Constr *Awaiting Model Update*>,
 'OIL3': <gurobi.Constr *Awaiting Model Update*>,
 'VEG1': <gurobi.Constr *Awaiting Model Update*>,
 'VEG2': <gurobi.Constr *Awaiting Model Update*>}

Moreover, there are some constraints which are implicitly given. (They may not sound really a "resource" of the company intuitively, nevertheless serve as constraints of the LP model.)
- The balance between buy, refine, store. (Last month Store + This month Buy = This month Refine + This month Store)
$$ S_{month - 1, oil} + B_{month, oil} = R_{month, oil} + S_{month, oil} $$

In [14]:
# previous month storage + buy = refine + store
c33 = (
    store[time_periods[time_periods.index(time_period) - 1], oil]
    + buy[time_period, oil] ==
    refine[time_period, oil] + store[time_period, oil]
    for oil in oils
    for time_period in time_periods if time_period != time_periods[0]
)
m.addConstrs(c33, "StoreBalance")

{('OIL1', 'Apr'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'Feb'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'Jun'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'Mar'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL1', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL2', 'Apr'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL2', 'Feb'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL2', 'Jun'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL2', 'Mar'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL2', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL3', 'Apr'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL3', 'Feb'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL3', 'Jun'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL3', 'Mar'): <gurobi.Constr *Awaiting Model Update*>,
 ('OIL3', 'May'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG1', 'Apr'): <gurobi.Constr *Awaiting Model Update*>,
 ('VEG1', 'Feb'): <gurobi.Constr *Awaiting Model Update*

- The balance between refine (5 oils) and final product. (Produce 500 → Refine 5 kinds of oils also sum up to 500)
$$ P_{month - 1, oil} + B_{month, oil} = R_{month, oil} + S_{month, oil} $$

In [15]:
c4 = (
    refine.sum(time_period) == produce[time_period]
    for time_period in time_periods
)
m.addConstrs(c4, "Conserve")

{'Apr': <gurobi.Constr *Awaiting Model Update*>,
 'Feb': <gurobi.Constr *Awaiting Model Update*>,
 'Jan': <gurobi.Constr *Awaiting Model Update*>,
 'Jun': <gurobi.Constr *Awaiting Model Update*>,
 'Mar': <gurobi.Constr *Awaiting Model Update*>,
 'May': <gurobi.Constr *Awaiting Model Update*>}

Finally, what is the objective the company wants to achieve? Of course it is a profit which should be maximized. How the profit is determined?

$$Total Profit = SalesRevenue - CostBuying - CostStoring$$


In [16]:
priceStore = 5
saleprice = 150
prices = [110, 120, 130, 110, 115,
          130, 130, 110, 90, 115,
          110, 140, 130, 100, 95,
          120, 110, 120, 120, 125,
          100, 120, 150, 110, 105,
          90, 100, 140, 80, 135]
prices = dict(zip(list(it.product(time_periods, oils)), prices))

# sales
sales = saleprice * produce.sum()
# cost for purchasing raw oil (buy)
cost_purchase = buy.prod(prices)
# cost for storing
cost_store = priceStore * store.sum()
objective_func = sales - cost_purchase - cost_store
m.setObjective(objective_func, GRB.MAXIMIZE)

In [17]:
# optimize and result
m.optimize()
for v in m.getVars():
    if v.X != 0:
        print(v.VarName, ":", v.X)

Optimize a model with 65 rows, 96 columns and 258 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+02]
Presolve removed 28 rows and 45 columns
Presolve time: 0.01s
Presolved: 37 rows, 51 columns, 149 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7425000e+05   1.065625e+03   0.000000e+00      0s
      28    1.0784259e+05   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.02 seconds
Optimal objective  1.078425926e+05
Buy[Feb,OIL2] : 250.0
Buy[May,OIL3] : 500.0
Buy[Jun,VEG1] : 659.2592592592592
Buy[Jun,VEG2] : 540.7407407407408
Buy[Jun,OIL2] : 750.0
Refine[Jan,VEG1] : 85.18518518518519
Refine[Jan,VEG2] : 114.81481481481481
Refine[Jan,OIL3] : 250.0
Refine[Feb,VEG2] : 200.0
Refine[Feb,OIL3] : 250.0
Refine[Mar,VEG1] : 159.25925925925924
Refine[Mar,VEG2] : 40.74074074074077
Refine[Mar,OIL2] : 250.0
Refine[Apr,VEG1] : 15

## Entire Code

In [18]:
# Set parameters
oils = ["VEG1", "VEG2", "OIL1", "OIL2", "OIL3"]
time_periods = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]

vegLim = 200
oilLim = 250

hardness = [8.8, 6.1, 2.0, 4.2, 5.0]
hardness = dict(zip(oils, hardness))
hardUpper = 6
hardLower = 3

initStore = 500
LastStore = initStore

priceStore = 5
saleprice = 150
prices = [110, 120, 130, 110, 115,
          130, 130, 110, 90, 115,
          110, 140, 130, 100, 95,
          120, 110, 120, 120, 125,
          100, 120, 150, 110, 105,
          90, 100, 140, 80, 135]
prices = dict(zip(list(it.product(time_periods, oils)), prices))

# instantiate model
m = Model("Oil_food")

# set variables with using subscript
buy = m.addVars(time_periods, oils, name = "Buy")
refine = m.addVars(time_periods, oils, name = "Refine")
produce = m.addVars(time_periods, name = "Produce")
store = m.addVars(time_periods, oils, name = "Store")

# set constraints
c11 = (
    quicksum(refine[time_period, oil] for oil in oils if "VEG" in oil)
    <= vegLim for time_period in time_periods
)
c12 = (
    quicksum(refine[time_period, oil] for oil in oils if "OIL" in oil)
    <= oilLim for time_period in time_periods
)
m.addConstrs(c11, "vegLim")
m.addConstrs(c12, "oilLim")

c21 = (
    quicksum(hardness[oil] * refine[time_period, oil] for oil in oils) 
    >= hardLower*produce[time_period] for time_period in time_periods
)
c22 = (
    quicksum(hardness[oil] * refine[time_period, oil] for oil in oils) 
    <= hardUpper*produce[time_period] for time_period in time_periods
)
m.addConstrs(c21, "hardLower")
m.addConstrs(c22, "hardUpper")

c31 = (initStore + buy[time_periods[0], oil] ==
     refine[time_periods[0], oil] + store[time_periods[0], oil]
     for oil in oils)
c32 = (
    store[time_periods[-1], oil]  == lastStore
    for oil in oils
)
m.addConstrs(c31, "initStore")
m.addConstrs(c32, "lastStore")

c33 = (
    store[time_periods[time_periods.index(time_period) - 1], oil]
    + buy[time_period, oil] ==
    refine[time_period, oil] + store[time_period, oil]
    for oil in oils
    for time_period in time_periods if time_period != time_periods[0]
)
m.addConstrs(c33, "StoreBalance")

c4 = (
    refine.sum(time_period) == produce[time_period]
    for time_period in time_periods
)
m.addConstrs(c4, "Conserve")

# Optimize
sales = saleprice * produce.sum()
cost_purchase = buy.prod(prices)
cost_store = priceStore * store.sum()
objective_func = sales - cost_purchase - cost_store

m.setObjective(objective_func, GRB.MAXIMIZE)
m.optimize()
for v in m.getVars():
    if v.X != 0:
        print(v.VarName, ":", v.X)

Optimize a model with 65 rows, 96 columns and 258 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+02]
Presolve removed 28 rows and 45 columns
Presolve time: 0.02s
Presolved: 37 rows, 51 columns, 149 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7425000e+05   1.065625e+03   0.000000e+00      0s
      28    1.0784259e+05   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.04 seconds
Optimal objective  1.078425926e+05
Buy[Feb,OIL2] : 250.0
Buy[May,OIL3] : 500.0
Buy[Jun,VEG1] : 659.2592592592592
Buy[Jun,VEG2] : 540.7407407407408
Buy[Jun,OIL2] : 750.0
Refine[Jan,VEG1] : 85.18518518518519
Refine[Jan,VEG2] : 114.81481481481481
Refine[Jan,OIL3] : 250.0
Refine[Feb,VEG2] : 200.0
Refine[Feb,OIL3] : 250.0
Refine[Mar,VEG1] : 159.25925925925924
Refine[Mar,VEG2] : 40.74074074074077
Refine[Mar,OIL2] : 250.0
Refine[Apr,VEG1] : 15